<p style="text-align:center">
        <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="Skills Network Logo">
</p>


### Analyse search terms on the e-commerce web server


##### In this assignment you will download the search term data set for the e-commerce web server and run analytic queries on it.


In [ ]:
# Install spark

In [5]:
!pip install pyspark
!pip install findspark
!pip install requests

In [ ]:
# Start session

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Saving and Loading a SparkML Model").getOrCreate()

23/02/15 14:33:56 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/15 14:33:58 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
# Download The search term dataset from the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

In [6]:
import requests
req = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv')
contents = req.content
src = open('searchterms.csv', 'wb')
src.write(contents)
src.close()

In [ ]:
# Load the csv into a spark dataframe

In [7]:
src = 'searchterms.csv'
search_term_dataset = spark.read.csv(src)

In [ ]:
# Print the number of rows and columns
# Take a screenshot of the code and name it as shape.jpg)

In [13]:
row_count = search_term_dataset.count()
col_count = len(search_term_dataset.columns)
print(f'Row count for search_term_dataset: {row_count}')
print(f'Column count for search_term_dataset: {col_count}')

Row count for search_term_dataset: 10001
Column count for search_term_dataset: 4


In [ ]:
# Print the top 5 rows
# Take a screenshot of the code and name it as top5rows.jpg)

In [14]:
search_term_dataset.show(5)

+---+-----+----+--------------+
|_c0|  _c1| _c2|           _c3|
+---+-----+----+--------------+
|day|month|year|    searchterm|
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
+---+-----+----+--------------+
only showing top 5 rows



In [ ]:
# Find out the datatype of the column searchterm?
# Take a screenshot of the code and name it as datatype.jpg)

In [19]:
print(search_term_dataset.schema['_c3'].dataType)

StringType


In [ ]:
# How many times was the term `gaming laptop` searched?
# Take a screenshot of the code and name it as gaminglaptop.jpg)

In [21]:
search_term_dataset.createOrReplaceTempView('searches')
spark.sql('SELECT COUNT(*) FROM searches WHERE _c3="gaming laptop"').show()

+--------+
|count(1)|
+--------+
|     499|
+--------+



In [ ]:
# Print the top 5 most frequently used search terms?
# Take a screenshot of the code and name it as top5terms.jpg)

In [31]:
spark.sql('SELECT _c3, COUNT(_c3) FROM searches GROUP BY _c3 ORDER BY COUNT(_c3) DESC').show(5)

[Stage 28:================================================>    (182 + 12) / 200]

+-------------+----------+
|          _c3|count(_c3)|
+-------------+----------+
|mobile 6 inch|      2312|
|    mobile 5g|      2301|
|mobile latest|      1327|
|       laptop|       935|
|  tablet wifi|       896|
+-------------+----------+
only showing top 5 rows



In [ ]:
# The pretrained sales forecasting model is available at  the below url
# https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

In [ ]:
# Load the sales forecast model.
# Take a screenshot of the code and name it as loadmodel.jpg)

In [35]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
!tar -xvzf model.tar.gz

--2023-02-15 15:28:11--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2023-02-15 15:28:11 (14.5 MB/s) - ‘model.tar.gz’ saved [1490/1490]

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sal

In [49]:
sales_prediction_parquet = 'sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet'
sales_prediction_df = spark.read.parquet(sales_prediction_parquet)
sales_prediction_df.printSchema()
sales_prediction_df.show()

root
 |-- intercept: double (nullable = true)
 |-- coefficients: vector (nullable = true)
 |-- scale: double (nullable = true)

+-------------------+-------------------+-----+
|          intercept|       coefficients|scale|
+-------------------+-------------------+-----+
|-13019.989140447298|[6.522567861288859]|  1.0|
+-------------------+-------------------+-----+



In [37]:
from pyspark.ml.regression import LinearRegressionModel
sales_prediction_model = LinearRegressionModel.load('sales_prediction.model')

In [ ]:
# Using the sales forecast model, predict the sales for the year of 2023.
# Take a screenshot of the code and name it as forecast.jpg

In [46]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
def predict(year):
    assembler = VectorAssembler(inputCols=["year"],outputCol="features")
    data = [[year,0]]
    columns = ["year", "sales"]
    _ = spark.createDataFrame(data, columns)
    __ = assembler.transform(_).select('features','sales')
    predictions = sales_prediction_model.transform(__)
    predictions.select('prediction').show()
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+



23/02/15 16:03:26 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/15 16:03:26 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
